In [205]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [206]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90.csv'

In [207]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)

In [208]:
crono_df.head(1)

,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Servicio,Semana,Cantidad Presupuestada,Doctor,Explicado
0,Dentix Calle 90,13/08/2018,BG051872,Nº 3784 - LEIDY JOHANA LOPEZ MONTES,PROFILAXIS. Pieza: 11,1,1,JOHANNA LISSETE MENDOZA DUARTE,SANDRA PATRICIA GONZALEZ TOVAR


In [209]:
PPE.head(1)

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,PROFILAXIS. Pieza: 11,Dentix Calle 90,25/08/2018,BG05192,Nº 4039 - ROSA HELENA LINDARTE,10,3165233002,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,MONICA PATRICIA PEREZ ZAMORA,40000,0,0.00%,0.00%,0,36000.0,27/08/2018,Sin Cita Producida


In [210]:
print('Numero de Procedimientos del PPE:',pptos_faltantes.count()[0])

Numero de Procedimientos del PPE: 0


In [211]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.
#Esta revision se hace a diario para hacer la programación de lo del día con lo antiguo q se vaya aceptando.
#la fecha de los pptos antiguos deben actualizarse a la fecha de hoy.

reporte = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
reporte['Cantidad Presupuestada'] = (reporte['Producidos']+reporte['Pendientes Producción'])/reporte['Importe Pres.']
reporte['Cantidad Presupuestada'] = np.ceil(reporte['Cantidad Presupuestada'])
reporte['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida
# Parece q la Cantidad Pendiente es innecesaria

pptos_faltantes = reporte[reporte['Semana'].isnull()]

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 'Servicio', 'Semana', 'Cantidad Presupuestada', 'Doctor_x', 'Explicado_x']]
print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes = pptos_faltantes.astype({'Cantidad Presupuestada': int})

pptos_faltantes.to_csv('pptos_faltantes.csv', index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes

Numero de tratamientos faltantes por crono: 0
